In [1]:
import requests
import urllib2
from bs4 import BeautifulSoup
import csv

In [2]:
URL_TEAM = "https://www.basketball-reference.com/play-index/event_finder.cgi?request=1&player_id=&id_val=&event_code=ast&year_id=2018&is_playoffs=N&team_id="
URL_OFFSET = "&opp_id=&quarter=&time_remain_quarter=&margin=&score=&offset="

# usage, URL_TEAM + "TEAM_NAME" + URL_OFFSET + "INDEX"

In [3]:
def getSoupFromURL(url):

    try:
        r = requests.get(url)
    except:
        return None

    return BeautifulSoup(r.text, "html5lib")

def getUrlFromTeamOffset(team, offset):
    return URL_TEAM + team + URL_OFFSET + str(offset)

In [4]:
nba_teams = ["ATL", "NJN", "BOS", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", 
             "MEM", "MIA", "MIL", "MIN", "NOH", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", 
             "UTA", "WAS"]

In [5]:
def get_all_plays_in_table(offset, team_name, data):
    url = getUrlFromTeamOffset(team_name, offset)
    soup = getSoupFromURL(url)
    table = soup.find('table', attrs={"class" : "sortable"})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td', attrs={"data-stat" : "play_desc"})
        for elem in cols:
            if elem.text.strip() == " ":
                continue
            string =  elem.text.strip()
            makes_split = string.split("makes")
            scorer = makes_split[0].strip()
            assist_by_split = string.split("assist by")
            passer = assist_by_split[1].strip()[:-1]
            shot_type = makes_split[1].split("shot")[0].strip()
            distance = 0
            if "at rim" not in string:
                distance = int(string.split("from")[1].split("ft")[0].strip())
        qtr_cols = row.find_all('td', attrs={"data-stat" : "quarter"})
        for elem in qtr_cols:
            if elem.text.strip() == " ":
                continue
            string = elem.text.strip()
            quarter = string[0]
            
        time_cols = row.find_all('td', attrs={"data-stat" : "time_remain_quarter"})
        for elem in time_cols:
            if elem.text.strip() == " ":
                continue
            string = elem.text.strip()
            splitted = string.split(":")
            minutes = int(splitted[0])
            seconds = int(splitted[1].split(".")[0])
            
        date_cols = row.find_all('td', attrs={"data-stat" : "date_game"})
        for elem in date_cols:
            if elem.text.strip() == " ":
                continue
            string_date = elem.text.strip()
            splitted = string_date.split("-")
            date = splitted[0] + splitted[1] + splitted[2]
        data.append([passer, scorer, shot_type, distance, quarter, minutes, seconds, date])
            
    return data

In [6]:
def get_number_of_urls(team_name):
    """
    Offset doesn't matter here
    """
    url = getUrlFromTeamOffset(team_name, 0)
    soup = getSoupFromURL(url)
    table = soup.find('table', attrs={"class" : "stats_table"})
    table_body = table.find('tbody')
    indexes = []
    rows = table_body.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        for elem in cols:
            if elem.text.strip() == " ":
                continue
            text = elem.text.strip()
            indexes.append(int(text.split("-")[0])-1)
    return indexes
    

In [7]:
nba_teams = ["HOU"]
for team in nba_teams:
    
    data = []
    indexes = get_number_of_urls(team)
    for index in indexes:
        get_all_plays_in_table(index, team, data)
    
    if team == "NJN":
        team = "BKN"
    elif team == "NOH":
        team = "NOP"
    file_name = "../../Data/assist_chart/" + team + ".csv"
    with open(file_name, "wb") as f:
        writer = csv.writer(f)
        writer.writerow(['Passer', 'Scorer', 'ShotType', 'Distance', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'GAME_DATE'])
        writer.writerows(data)
